## Use this file to compile the Stable Diffusion model using the AITemplate 

Alternately you can follow the method from AItemplate documented here at https://github.com/facebookincubator/AITemplate under the section docker install

This below section uses the same technique except that it uses the DJL container to compile the model and so we do not need to create or download any other docker image. However this file assumes you have ECR login permissions to pull the DJL container image

### Docker change data directory

This is needed to ensure docker can utilize the mounted EBS volume

In [ ]:
!ls -alrt /var/lib/docker

In [ ]:
%%sh
#!/bin/bash

rm -rf /home/ec2-user/SageMaker/docker_data_ait

sudo service docker stop

sudo mv /var/lib/docker /home/ec2-user/SageMaker/docker_data_ait


sudo ln -s /home/ec2-user/SageMaker/docker_data_ait /var/lib/docker

ls -alrt /var/lib/docker

sudo service docker start

### Clone the AITemplate 

under /home/ec2-user/SageMaker/compiled_ai_repo


we will need to install GIT LFS needed to clone the repo. If you have the LFS installed already then run the git clone recursive command only

In [ ]:
%%sh

curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash

sudo yum install git-lfs

git lfs install

mkdir -p /home/ec2-user/SageMaker/compiled_ai_repo/
cd /home/ec2-user/SageMaker/compiled_ai_repo/
git clone https://github.com/lanking520/AITemplate --recursive



### Over write the download file to remove the token 


In [ ]:
%%writefile /home/ec2-user/SageMaker/compiled_ai_repo/AITemplate/examples/05_stable_diffusion/scripts/download_pipeline.py
#  Copyright (c) Meta Platforms, Inc. and affiliates.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
import click
import torch
from diffusers import StableDiffusionPipeline


@click.command()
@click.option("--token", default="", help="access token")
@click.option(
    "--save_directory",
    default="./tmp/diffusers-pipeline/stabilityai/stable-diffusion-v2",
    help="pipeline files local directory",
)
def download_pipeline_files(token, save_directory) -> None:
    StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base",
        revision="fp16",
        torch_dtype=torch.float16,
    ).save_pretrained(save_directory)


if __name__ == "__main__":
    download_pipeline_files()

### Build and install docker image  AITemplate with DJL container 

we wil use the DJL container to compile the weights

In [ ]:
%%sh

# Get the region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}



djl_image_uri="763104351884.dkr.ecr.${region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117"
login_img_uri=763104351884.dkr.ecr.${region}.amazonaws.com

echo $region
echo $djl_image_uri
echo $inference_image_uri
echo "image to used for compilations is $djl_image_uri"

aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin $login_img_uri

docker pull $djl_image_uri

docker tag $djl_image_uri djl-ait-sd

docker images # this should list djl-ait-sd which can be used to login into and use as below 



### Now compile and create the AIT weights

#### To compile the weights 

In [ ]:
%%sh

cd /home/ec2-user/SageMaker/compiled_ai_repo
sudo rm -rf tmp

# to delete any previous weights
cd /home/ec2-user/SageMaker/default_ai_repo/AITemplate/examples/05_stable_diffusion/scripts
sudo rm -rf tmp



In [ ]:
%%writefile /home/ec2-user/SageMaker/compiled_ai_repo/generate_weights.sh

pip3 install click
cd /model_repository/AITemplate/examples/05_stable_diffusion/scripts 

# Compile models
# -- this downloads and saves as diffusers-pipeline/ under /model_repository/AITemplate/examples/05_stable_diffusion/scripts/tmp
# - this gets mapped too -- /home/ec2-user/SageMaker/compiled_ai_repo/AITemplate/examples/05_stable_diffusion/scripts/tmp
python3 download_pipeline.py

# - that is why --local-dir",default="./tmp/diffusers-pipeline/stabilityai/stable-diffusion-v2"  -- works
# -- And all the weights are correctly under the ./tmp folder --
# -- which is mapped too --  /home/ec2-user/SageMaker/default_ai_repo/AITemplate/examples/05_stable_diffusion/scripts/tmp
python3 compile.py --local-dir ./tmp/diffusers-pipeline/stabilityai/stable-diffusion-v2 

# -- use this if we were to GIT clone and use that instead -- but this creates a huge model weights size
# python3 compile.py --local-dir /model_repository/stable-diffusion-2-1/


In [ ]:
%%sh

# for DJL container -- use this

cd /home/ec2-user/SageMaker/compiled_ai_repo

docker run --gpus=all --shm-size=4G --rm -p8000:8000 -p8001:8001 -p8002:8002 -v$(pwd):/model_repository djl-ait-sd /bin/bash /model_repository/generate_weights.sh

### Copy the weights to S3 location

old weights saved under s3://sagemaker-us-east-1-425576326687/raw_weights/stabilityai_working/aitemplate_g5/

In [ ]:
%%sh

ls -alrt /home/ec2-user/SageMaker/compiled_ai_repo/AITemplate/examples/05_stable_diffusion/scripts/tmp

cd /home/ec2-user/SageMaker/compiled_ai_repo/AITemplate/examples/05_stable_diffusion/scripts/tmp

# Get the region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

account=$(aws sts get-caller-identity --query "Account" --output text)

echo $account
echo $region

aws s3 rm s3://sagemaker-${region}-${account}/raw_weights/stabilityai/aitemplate_g5/ --recursive
aws s3 cp . s3://sagemaker-${region}-${account}/raw_weights/stabilityai/aitemplate_g5/ --recursive 

aws s3 ls s3://sagemaker-${region}-${account}/raw_weights/stabilityai/aitemplate_g5/

In [ ]:
print(
    " please check your S3 location to ensure the weights have been copied to your S3"
)